In [1]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from math import *
import tensorflow as tf

from _context import dl_uncertainty
import dl_uncertainty.tf_utils as tf_utils
import dl_uncertainty.tf_utils.layers as layers

In [2]:
x = tf.constant(np.array([[1,2],[3,4.0]]), dtype=tf.float32)
sess = tf.Session()

### Sampling random operations with no variables

In [3]:
@layers.scoped
def dropout(x):
    return tf.nn.dropout(x, keep_prob=0.5)

ys = layers.sample(lambda x: dropout(x, name='do'), x, 8)
y_mean = tf.reduce_mean(ys, 0)  # should approximate x
ys2 = layers.sample(lambda x: dropout(x, name='do'), x, 8, examplewise=True)

In [4]:
sess.run(tf.global_variables_initializer())

print('x')
print(sess.run(x))

print('')
print('dropout(x) sampling ([n,N,...]) (first 3 samples)')
s = sess.run(ys)
print(s[:3], '...')
print('shape:', s.shape)

print('')
print('Sample mean (estimation of x)')
m = sess.run(y_mean)
print(m)

print('')
print('Example-wise dropout(x) sampling (first 2 elements)')
s = sess.run(ys2)
print(s[:2], '...')
print('shape:', s.shape)

x
[[1. 2.]
 [3. 4.]]

dropout(x) sampling ([n,N,...]) (first 3 samples)
[[[2. 0.]
  [0. 8.]]

 [[2. 0.]
  [0. 8.]]

 [[2. 0.]
  [0. 0.]]] ...
shape: (8, 2, 2)

Sample mean (estimation of x)
[[0.5  2.  ]
 [2.25 1.  ]]

Example-wise dropout(x) sampling (first 2 elements)
[[[2. 4.]
  [0. 0.]
  [0. 4.]
  [2. 4.]
  [2. 4.]
  [2. 0.]
  [2. 4.]
  [0. 0.]]

 [[0. 8.]
  [6. 8.]
  [0. 0.]
  [6. 8.]
  [6. 8.]
  [6. 8.]
  [6. 8.]
  [0. 8.]]] ...
shape: (2, 8, 2)


### Sampling operations with variables

In [5]:
def init_func(*args, **kwargs): return tf.random_uniform([])

@layers.scoped
def rando(x):
    v = tf.get_variable("var", initializer=tf.random_uniform([]))
    #tf.get_variable_scope().reuse_variables()
    print(v)
    return tf.random_uniform([])*v*tf.identity(x)

In [6]:
#trando = tf.make_template('rando', rando)
rr = [tf.map_fn(rando, x) for _ in range(2)]
samp = layers.sample(rando, x, 100)

<tf.Variable 'var:0' shape=() dtype=float32_ref>


ValueError: Variable var already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Users\Ivan\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1269, in __init__
    self._traceback = _extract_stack()
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2506, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Users\Ivan\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [7]:
sess.run(tf.global_variables_initializer())

rr_ = sess.run(rr)
for a in rr_:
    print(a)

InvalidArgumentError: The node 'var/Assign' has inputs from different frames. The input 'map/while/random_uniform' is in frame 'map/while/map/while/'. The input 'var' is in frame ''.

In [8]:
# samples should be different because outputs come from different subgraphs with different values of "var"
assert np.any(rr_[0] != rr_[1])

NameError: name 'rr_' is not defined

In [ ]:
samp_ = sess.run(samp)
print(samp_)

In [ ]:
# samples should be equal because outputs come from the same subgraph with a single "var"
assert np.all(samp_[0]==samp_[1])  # samples should be different

In [ ]:
v = tf.get_variable("var", initializer=tf.random_uniform(shape=tf.shape(x)))

@layers.scoped
def wrapv(fn, x):
    with tf.control_dependencies([tf.assign(v, x)]):
        return fn(v)
samp = layers.sample(lambda x: wrapv(rando, x), x, 10)

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(samp)